In [1]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import uuid
import difflib
import html5lib

In [2]:
player_index = pd.read_csv('../updated_datasets/player_index.csv')
player_index = player_index[player_index.columns[1:]]
player_index

,Player,UUID
0,Kevin Garnett,4035d838-5ed7-4507-86ad-0488a9b9c358
1,Shaquille O'Neal,74b99a51-bb65-436f-8bcc-7022af2a490a
2,Alonzo Mourning,8051eb29-ecd7-46b5-b2fa-415777e11e83
3,Juwan Howard,5bdd7fb5-050d-4b0c-befb-5fbf7109397c
4,Hakeem Olajuwon,be9d9c53-d445-4b8c-acc9-ad1c30e8598b
...,...,...
2224,Jordan Schakel,49852ad9-c5e3-4124-bd75-2ad27b5a859f
2225,Jordan Goodwin,70d9ad35-fb19-497a-a358-d77a2a2e89ba
2226,Craig Sword,c29acdbe-9077-44c0-a8b3-f997ddff72a2
2227,Jaime Echenique,727da240-fbce-4643-ba94-f74509e2e746


In [3]:
year = '2022'
draft_df = pd.read_html(requests.get(f'https://www.basketball-reference.com/draft/NBA_{year}.html').text)[0]
draft_df.columns = draft_df.columns.get_level_values(1)
draft_df = draft_df[draft_df.columns[1:4]]
draft_df.dropna(inplace=True)
draft_df = draft_df[draft_df['Player'] != 'Player']
draft_df.rename(columns={'Pk':'Pick','Tm':'Team'}, inplace=True)
draft_df['Year'] = year
draft_df.head()

,Pick,Team,Player,Year
0,1,ORL,Paolo Banchero,2022
1,2,OKC,Chet Holmgren,2022
2,3,HOU,Jabari Smith Jr.,2022
3,4,SAC,Keegan Murray,2022
4,5,DET,Jaden Ivey,2022


In [9]:
table_list = []
df_list = []
null_list = []
def draft_data(num):
    for i in range(num):
        year = 2000 + i
        var_name = f'draft_{year}'
        table_list.append(var_name)
        
        #grab the datset
        globals()[var_name] = pd.read_html(requests.get(f'https://www.basketball-reference.com/draft/NBA_{year}.html').text)[0]
        
        #clean the data
        globals()[var_name].columns = globals()[var_name].columns.get_level_values(1)
        globals()[var_name] = globals()[var_name][globals()[var_name].columns[1:4]]
        globals()[var_name].dropna(inplace=True)
        globals()[var_name] = globals()[var_name][globals()[var_name]['Player'] != 'Player']
        globals()[var_name].rename(columns={'Pk':'Pick','Tm':'Team'}, inplace=True)
        globals()[var_name]['Year'] = year

        #gather uuids
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        
        #locate mispelled names
        null_players = []
        for row in globals()[var_name][globals()[var_name]['UUID'].isnull()]['Player']:
            null_players.append(row)
        null_players = pd.DataFrame(null_players, columns=['Player']).drop_duplicates()
        
        #locate correct names from index
        nearest_name = []
        original_name = []
        for name in null_players['Player']:
            close_matches = difflib.get_close_matches(name, player_index['Player'], 1)
            for x in close_matches:
                    nearest_name.append(x)
                    original_name.append(name)
        nearest_name = pd.DataFrame(nearest_name, columns=['New_name'])
        nearest_name['Original_name'] = original_name
        
        #fix mispelled names 
        for bad_name in globals()[var_name]['Player']:
            if bad_name in [x for x in nearest_name['Original_name']]:
                globals()[var_name].loc[globals()[var_name][globals()[var_name]['Player'] == bad_name].index, 'Player'] = [
                    x for x in nearest_name[nearest_name['Original_name'] == bad_name]['New_name']][0]
        
        #merge w/ index table to add unique ids and save csvs
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        globals()[var_name].drop(columns=['UUID_x'], inplace=True)
        globals()[var_name].rename(columns={'UUID_y':'UUID'}, inplace=True)
                
        #clean remaining nulls
        globals()[var_name].dropna(inplace=True)
        
        globals()[var_name].to_csv(f'../updated_datasets/draft_data/{var_name}.csv')
        
        #add dataframe to my output list of dataframes
        df_list.append(globals()[var_name])

    return table_list

In [13]:
draft_data(22)

['draft_2000',
 'draft_2001',
 'draft_2002',
 'draft_2003',
 'draft_2004',
 'draft_2005',
 'draft_2006',
 'draft_2007',
 'draft_2008',
 'draft_2009',
 'draft_2010',
 'draft_2011',
 'draft_2012',
 'draft_2013',
 'draft_2014',
 'draft_2015',
 'draft_2016',
 'draft_2017',
 'draft_2018',
 'draft_2019',
 'draft_2020',
 'draft_2021']